In [2]:
import os
os.environ['ESOSKY'] = "/users/long/sdss/esosky/"

In [3]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table

from lvmdrp.core.constants import LVM_UNAM_URL
from lvmdrp.utils.examples import fetch_example_data
from lvmdrp.core import rss, image, sky, spectrum1d
from lvmdrp.functions import skyMethod
from multiprocessing import Pool, cpu_count

plt.style.use("seaborn-v0_8-talk")



In [4]:
os.environ['ESOSKY'] 

'/users/long/sdss/esosky/'

In [5]:
# define input data directory
data_path = os.path.abspath(os.path.join("..", "data"))

# let's create the output directory
output_path = "./data"
os.makedirs(output_path, exist_ok=True)

fetch_example_data(url=LVM_UNAM_URL, name="sim_1d", dest_path=data_path)

# define the target simulation
SIM_NAME = "DIG_bright_linear_full_900_{}.fits"
in_path = os.path.join(data_path, "sim_1d", SIM_NAME)
out_path = os.path.join(output_path, SIM_NAME)

[INFO]: example data already exists


In [ ]:
skyMethod.createMasterSky_drp(
    in_rss=out_path.format("sci"),
    out_sky=out_path.format("msci")
)
skyMethod.createMasterSky_drp(
    in_rss=out_path.format("skyw"),
    out_sky=out_path.format("mskyw")
)
skyMethod.createMasterSky_drp(
    in_rss=out_path.format("skye"),
    out_sky=out_path.format("mskye")
)

In [ ]:
os.environ['ESOSKY']

In [ ]:


skyMethod.sepContinuumLine_drp(
    sky_ref=out_path.format("msci"),
    out_cont_line=out_path.format("sci_cl"),
    method="skycorr",
    sky_sci=out_path.format("msci")
)


In [ ]:
skyMethod.sepContinuumLine_drp(
    sky_ref=out_path.format("mskyw"),
    out_cont_line=out_path.format("skyw_cl"),
    method="skycorr",
    sky_sci=out_path.format("msci")
)


In [ ]:

skyMethod.sepContinuumLine_drp(
    sky_ref=out_path.format("mskye"),
    out_cont_line=out_path.format("skye_cl"),
    method="skycorr",
    sky_sci=out_path.format("msci")
)

In [ ]:
cont_line = rss.RSS()
cont_line.loadFitsData(out_path.format("skye_cl"))

skye = rss.RSS()
skye.loadFitsData(out_path.format("mskye"))

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(25,7), sharex=True, sharey=False)
ax1.step(cont_line._wave, cont_line._data[0], lw=1, color="tab:blue")
ax2.step(skye._wave, skye._data, lw=1, color="0.5")
ax2.step(cont_line._wave, cont_line._data[1], lw=1, color="tab:red")
ax1.set_xlim(cont_line._wave.min(), cont_line._wave.max())
ax1.set_title("sky continuum", loc="left")
ax2.set_title("sky emission", loc="left")
ax1.set_ylabel("count (e-/pix)")
ax2.set_ylabel("count (e-/pix)")
ax2.set_xlabel("wavelength (A)")
fig.tight_layout()

In [ ]:
os.chdir('/Users/long/SDSS/t230720/examples.230720/sky_module')

In [6]:
skyMethod.evalESOSky_drp(
    sky_ref=out_path.format("msci"),
    out_rss=out_path.format("model_sci"),
    resample_method="linear", err_sim=50,
    parallel=0
)


[WARNING]: 'OBSERVAT' is not in reference sky header. Assuming OBSERVAT='LCO'
[WARNING]: 'OBSTIME' is not in reference sky header. Falling back to 'MJD'
[INFO]: trying skymodel from pre-computed airglow lines
[WARNING]: no suitable airglow spectrum found
[INFO]: calculating airglow lines with parameters airmass = 1.2, time = 1, season = 1, resol = 1000000 pwv = -1
[ERROR]: failed while running airglow lines calculations
[ERROR]: Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG
STOP  LBLRTM EXIT 
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG
STOP  LBLRTM EXIT 
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG
STOP  LBLRTM EXIT 
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG
STOP  LBLRTM EXIT 
Note: The f

The destination directory already exists /users/long/sdss/esosky/lib/skymodel/sm-01_mod2/data/lib


[INFO]: calculating effective atmospheric transmission
[ERROR]: failed while running effective atmospheric transmission calculations
[ERROR]: [ ERROR ] Dumping all 1 error(s):
[ ERROR ]   [1/1] 'A user-defined error: File opening failed: data/./lib/LBL_A10_t0_s0_w005_R1000000_T.fits' (34) at sm_linetrans_calclib:sm_linetrans.c:609

[INFO]: successfully finished 'calcskymodel'


Afterward, the destination directory  exists /users/long/sdss/esosky/lib/skymodel/sm-01_mod2/data/lib
Cannot found output/transspec.fits from dir /Users/long/SDSS/skymodel/sm-01_mod2


FileNotFoundError: [Errno 2] No such file or directory: 'output/transspec.fits'

In [ ]:
os.getcwd()

In [ ]:

os.chdir('/Users/long/SDSS/t230720/examples.230720/sky_module')

In [ ]:
skyMethod.evalESOSky_drp(
    sky_ref=out_path.format("mskyw"),
    out_rss=out_path.format("model_skyw"),
    resample_method="linear", err_sim=50,
    parallel=0
)


In [ ]:

skyMethod.evalESOSky_drp(
    sky_ref=out_path.format("mskye"),
    out_rss=out_path.format("model_skye"),
    resample_method="linear", err_sim=50,
    parallel=0
)